<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-import-and-preparation" data-toc-modified-id="Data-import-and-preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data import and preparation</a></span><ul class="toc-item"><li><span><a href="#Translating-column-names-to-English" data-toc-modified-id="Translating-column-names-to-English-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Translating column names to English</a></span></li></ul></li><li><span><a href="#Data-transformation-and-Feature-engineering" data-toc-modified-id="Data-transformation-and-Feature-engineering-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data transformation and Feature engineering</a></span><ul class="toc-item"><li><span><a href="#Creating-default-flag" data-toc-modified-id="Creating-default-flag-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Creating default flag</a></span></li><li><span><a href="#Calculating-key-KPIs" data-toc-modified-id="Calculating-key-KPIs-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Calculating key KPIs</a></span></li><li><span><a href="#Outputing-data-for-further-modeling" data-toc-modified-id="Outputing-data-for-further-modeling-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Outputing data for further modeling</a></span></li></ul></li><li><span><a href="#Exploring-client-characteristics" data-toc-modified-id="Exploring-client-characteristics-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Exploring client characteristics</a></span><ul class="toc-item"><li><span><a href="#Aggregating-monthly-data-to-form-client-characteristic" data-toc-modified-id="Aggregating-monthly-data-to-form-client-characteristic-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Aggregating monthly data to form client characteristic</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import random as random

# plotly standard imports
from datetime import datetime
from datetime import timedelta  

In [2]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [3]:
import tensorflow as tf
from tensorflow import keras

In [4]:
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot
cufflinks.go_offline()

# Set global theme
cufflinks.set_config_file(world_readable=True, theme='pearl')
import plotly.figure_factory as ff


import plotly.express as px
import plotly.graph_objs as go
import chart_studio.plotly as py

# Data import and preparation

In [5]:
data_input=pd.read_excel("raw_data.xlsx", sep=";")

In [6]:
dataset_downland_date=data_input.data_ts.max()
dataset_downland_date

Timestamp('2019-08-23 00:00:00')

In [7]:
data_input.columns

Index(['nrpożyczki', 'nr Klienta', 'płeć', 'wiek', 'spłacono dotychczas',
       'data ostatniej spłaty', 'kwota pozyczki', 'sposob wyplaty',
       'data wyplaty', ' Ilość rat', 'Pierwsza rata', 'kod pocztowy',
       'miejscowosc', 'kapital', 'odsetki', 'prowizja',
       'opłata administracyjna', 'oplata windykacyjne', 'inne', 'razem', 'DPD',
       'Pierwsza rata.1', 'rata', 'Uknown1', 'czy odstąpienie',
       'data odstapienia ', 'metoda podpisania umowy broker online', 'okres',
       'kwota', 'data', 'yield', 'Day', 'Month', 'Year', 'data_ts'],
      dtype='object')

In [8]:
data_input.head(10)

,nrpożyczki,nr Klienta,płeć,wiek,spłacono dotychczas,data ostatniej spłaty,kwota pozyczki,sposob wyplaty,data wyplaty,Ilość rat,...,data odstapienia,metoda podpisania umowy broker online,okres,kwota,data,yield,Day,Month,Year,data_ts
0,20180524/QS/4829,19086,Mezczyzna,27,0.0,0,3000,Przelew bankowy,2018-05-25,24,...,NaT,broker,1,0.0,NaN,0.0,NaN,NaN,NaN,NaT
1,20180524/QS/4829,19086,Mezczyzna,27,0.0,0,3000,Przelew bankowy,2018-05-25,24,...,NaT,broker,2,0.0,NaN,0.0,NaN,NaN,NaN,NaT
2,20180524/QS/4829,19086,Mezczyzna,27,0.0,0,3000,Przelew bankowy,2018-05-25,24,...,NaT,broker,3,0.0,NaN,0.0,NaN,NaN,NaN,NaT
3,20180524/QS/4829,19086,Mezczyzna,27,0.0,0,3000,Przelew bankowy,2018-05-25,24,...,NaT,broker,4,0.0,NaN,0.0,NaN,NaN,NaN,NaT
4,20180524/QS/4829,19086,Mezczyzna,27,0.0,0,3000,Przelew bankowy,2018-05-25,24,...,NaT,broker,5,0.0,NaN,0.0,NaN,NaN,NaN,NaT
5,20180524/QS/4829,19086,Mezczyzna,27,0.0,0,3000,Przelew bankowy,2018-05-25,24,...,NaT,broker,6,0.0,NaN,0.0,NaN,NaN,NaN,NaT
6,20180524/QS/4829,19086,Mezczyzna,27,0.0,0,3000,Przelew bankowy,2018-05-25,24,...,NaT,broker,7,0.0,NaN,0.0,NaN,NaN,NaN,NaT
7,20180524/QS/4829,19086,Mezczyzna,27,0.0,0,3000,Przelew bankowy,2018-05-25,24,...,NaT,broker,8,0.0,NaN,0.0,NaN,NaN,NaN,NaT
8,20180524/QS/4829,19086,Mezczyzna,27,0.0,0,3000,Przelew bankowy,2018-05-25,24,...,NaT,broker,9,0.0,NaN,0.0,NaN,NaN,NaN,NaT
9,20180524/QS/4829,19086,Mezczyzna,27,0.0,0,3000,Przelew bankowy,2018-05-25,24,...,NaT,broker,10,0.0,NaN,0.0,NaN,NaN,NaN,NaT


In [9]:
data_input["data"]= pd.to_datetime(data_input.data)
data_input["data_wyplaty"]= pd.to_datetime(data_input["data wyplaty"])
data_input['Pierwsza rata']= pd.to_datetime(data_input['Pierwsza rata'])
data_input["initial_loan_value"]=data_input.rata*data_input[' Ilość rat']

In [10]:
features=['nrpożyczki', 'nr Klienta', 'płeć', 'wiek','kod pocztowy','sposob wyplaty',
        'initial_loan_value',' Ilość rat',"data_wyplaty", 'Pierwsza rata','rata',  'okres',
       'kwota', 'data_ts', 'yield']

In [11]:
df=data_input[features]

In [12]:
df.columns

Index(['nrpożyczki', 'nr Klienta', 'płeć', 'wiek', 'kod pocztowy',
       'sposob wyplaty', 'initial_loan_value', ' Ilość rat', 'data_wyplaty',
       'Pierwsza rata', 'rata', 'okres', 'kwota', 'data_ts', 'yield'],
      dtype='object')

## Translating column names to English

In [13]:
features_translate={
    'nrpożyczki':'loan_ID',
    'nr Klienta':"client_ID",
    'sposob wyplaty':"cash_payment",
    'okres':"period",
    "rata":"installment",
    'kod pocztowy':"post_code",
    ' Ilość rat':"installment_count",
    'Pierwsza rata':"first_installment_date",
    'kwota':"payment_value",
    "data_ts":"payment_date",
    "data_wyplaty":"payout_date",
    "yield":"cumulated_payments",
    'płeć':"sex",
    "wiek":"age",

    


}

In [14]:
df.rename(columns=features_translate, inplace=True)

In [15]:
df["sex"]=np.where(df.sex=="Kobieta","Female","Male")

In [16]:
df.columns

Index(['loan_ID', 'client_ID', 'sex', 'age', 'post_code', 'cash_payment',
       'initial_loan_value', 'installment_count', 'payout_date',
       'first_installment_date', 'installment', 'period', 'payment_value',
       'payment_date', 'cumulated_payments'],
      dtype='object')

# Data transformation and Feature engineering

In [17]:
df["expected_payment_date"]=df.first_installment_date

In [18]:
import datetime
for i in range(0,len(df)):
    df["expected_payment_date"][i]=df.first_installment_date[i]+pd.DateOffset(months=int(df.period[i])-1)


In [19]:
df_grouped=df[(df.payment_date.isnull()==False)][['loan_ID','period',"payment_date",'cumulated_payments']].groupby('loan_ID').max()
df_grouped.rename(columns={'period':"last_payment_period","payment_date":"last_payment_date",'cumulated_payments':'max_cumulated_payments'}, inplace=True)
df_grouped.reset_index(inplace=True)
df=df.merge(df_grouped, on="loan_ID")

In [20]:
df.columns

Index(['loan_ID', 'client_ID', 'sex', 'age', 'post_code', 'cash_payment',
       'initial_loan_value', 'installment_count', 'payout_date',
       'first_installment_date', 'installment', 'period', 'payment_value',
       'payment_date', 'cumulated_payments', 'expected_payment_date',
       'last_payment_period', 'last_payment_date', 'max_cumulated_payments'],
      dtype='object')

## Creating default flag

Filling the next payment with 0s and predicted date to segment defaulting clients from clients expected to pay after dataset was downlanded

In [21]:
df.payment_date=np.where(df.period==df.last_payment_period+1,
                        df.last_payment_date+pd.DateOffset(months=2),
                        df.payment_date)

df.cumulated_payments=np.where(df.period==df.last_payment_period+1,
                        df.max_cumulated_payments,
                        df.cumulated_payments)

df["cash_payment"]=np.where(df.cash_payment=='Przelew bankowy',0,1)

Removing empty entries (no dates) and payments expected in future - after dataset download date

In [22]:
df=df[(df.payment_date.isnull()==False)].sort_values(by=["loan_ID","period"])
df.shape

(39029, 19)

In [23]:
df=df[df.payment_date<=dataset_downland_date]
df.shape

(37311, 19)

In [24]:
df.head(10)

,loan_ID,client_ID,sex,age,post_code,cash_payment,initial_loan_value,installment_count,payout_date,first_installment_date,installment,period,payment_value,payment_date,cumulated_payments,expected_payment_date,last_payment_period,last_payment_date,max_cumulated_payments
46170,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,1,244.68,2017-07-19,244.68,2017-07-20,13,2018-07-27,3194.68
46171,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,2,245.00,2017-08-22,489.68,2017-08-20,13,2018-07-27,3194.68
46172,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,3,245.00,2017-09-18,734.68,2017-09-20,13,2018-07-27,3194.68
46173,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,4,245.00,2017-10-20,979.68,2017-10-20,13,2018-07-27,3194.68
46174,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,5,245.00,2017-11-17,1224.68,2017-11-20,13,2018-07-27,3194.68
46175,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,6,245.00,2017-12-20,1469.68,2017-12-20,13,2018-07-27,3194.68
46176,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,7,245.00,2018-01-22,1714.68,2018-01-20,13,2018-07-27,3194.68
46177,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,8,245.00,2018-02-20,1959.68,2018-02-20,13,2018-07-27,3194.68
46178,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,9,245.00,2018-03-20,2204.68,2018-03-20,13,2018-07-27,3194.68
46179,20170626/13,7,Female,57,41-200,0,5872.32,24,2017-06-26,2017-07-20,244.68,10,245.00,2018-04-19,2449.68,2018-04-20,13,2018-07-27,3194.68


In [79]:

trace0=go.Scatter(
        y=df.head(100).age,
        x=df.head(100).cumulated_payments,
        mode='markers',
        marker=dict(
        color="red",
        size=5,
        opacity=0.3
        ),
       
    )








data=[trace0]
figure=go.Figure(
    data=data,
    layout=go.Layout(
        title="Mileage vs Age",
        yaxis=dict(title="Mileage"),
        xaxis=dict(title="Age")

    
    ))
iplot(figure)